# TP - Computação Natural
#### "Predict whether a mammogram mass is benign or malignant"

1. BI-RADS assessment: 1 to 5 (ordinal)  
2. Age: patient's age in years (integer)
3. Shape: mass shape: round=1 oval=2 lobular=3 irregular=4 (nominal)
4. Margin: mass margin: circumscribed=1 microlobulated=2 obscured=3 ill-defined=4 spiculated=5 (nominal)
5. Density: mass density high=1 iso=2 low=3 fat-containing=4 (ordinal)
6. Severity: benign=0 or malignant=1 (binominal)

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy import stats

## Get the Data

In [ ]:
data = pd.read_csv('mammographic_masses.data.txt')
data

** Convert missing data (indicated by a ?) into NaN and add the appropriate column names (BI_RADS, age, shape, margin, density, and severity) **

In [ ]:
data = data.replace('?',np.nan)
data.columns = ['BI_RADS','Age','Shape','Margin','Density','Severity']
data

** Drop BI_RADS column because it has no influence on the severity forecast **

In [ ]:
data = data.drop(columns=['BI_RADS'])

** Convert datatype 'object' to 'float64' **  

In [ ]:
data.info()

In [ ]:
data = data.astype(float)
data

In [ ]:
data.info()

In [ ]:
data.describe()

### Analysing missing values
**First we get the missing values per feature.**

*Lets check them out as well*

In [ ]:
missing_values_feature = data.isnull().sum(axis=0)
graph = missing_values_feature.drop(labels='Severity')
graph

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(131)
plt.bar(graph.axes[0].to_list(), graph.values)

*We then develop a heatmap to give us some more information*

In [ ]:
sns.heatmap(data.isnull(),yticklabels=False,cbar=False,cmap='viridis')

*Finally we can check the percentage of missing values per feature*

In [ ]:
percent_missing = data.isnull().sum() * 100 / len(data)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
print(missing_value_df)

**After analysing the columns, we should have a look at the rows**

In [ ]:
data_missing = len(data.columns) - (data.apply(lambda x: x.count(), axis=1))
missing_values_data_rows = pd.DataFrame({'data_missing':data_missing})
missing_values_data_rows.sort_values('data_missing',inplace=True,ascending=False)
missing_values_data_rows

**Now lets analyse the missing data per class (Severity = 0 or Severity = 1).**

*First we group the missing values per class*

In [ ]:
grouped_data = data.groupby('Severity')
missing_values_class = grouped_data.count().rsub(grouped_data.size(), axis=0)
missing_values_class

*Now we split the dataframe per class so we can draw our plot*

In [ ]:
m_new_1, m_new_2 = missing_values_class.head(1), missing_values_class.tail(1)

In [ ]:
x = np.arange(len(m_new_1.axes[1].to_list()))
width = 0.4

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, m_new_1.values[0], width=width, label = "Severity 0")
rects2 = ax.bar(x + width/2, m_new_2.values[0], width=width, label = "Severity 1")

ax.set_xticks(x)
ax.set_xticklabels(m_new_1.axes[1].to_list())
ax.legend()

**Finally, for each class we're going to calculate the number of rows that have 1 and 2 NaN values**

In [ ]:
rows_mv1_sv0 = 0
rows_mv2_sv0 = 0
rows_mv1_sv1 = 0
rows_mv2_sv1 = 0

In [ ]:
for index, row in data.iterrows():
    if(row['Severity'] == 0):
        if(row.isnull().sum() == 1):
            rows_mv1_sv0 += 1
        elif(row.isnull().sum() == 2):
            rows_mv2_sv0 += 1
    else:
        if(row.isnull().sum() == 1):
            rows_mv1_sv1 += 1
        elif(row.isnull().sum() == 2):
            rows_mv2_sv1 += 1

*We create a dataframe only for visualization purpose*

In [ ]:
numberofnan_class = pd.DataFrame(np.array([[rows_mv1_sv0,rows_mv2_sv0], [rows_mv1_sv1,rows_mv2_sv1]]), 
                                    index=['Severity 0','Severity 1'], columns=['1 NaN', '2 NaN'])
numberofnan_class

In [ ]:
labels = ['1 NaN', '2 NaN']
x = np.arange(len(labels))
width = 0.4

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, [rows_mv1_sv0,rows_mv2_sv0], width=width, label = "Severity 0")
rects2 = ax.bar(x + width/2, [rows_mv1_sv1,rows_mv2_sv1], width=width, label = "Severity 1")

ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

*With this information we can also see the number of rows with 1 or 2 missing values per class*

In [ ]:
nan_class = pd.DataFrame(np.array([[rows_mv1_sv0+rows_mv2_sv0], [rows_mv1_sv1+rows_mv2_sv1]]), 
                                    index=['Severity 0','Severity 1'], columns=['Sum'])
nan_class

In [ ]:
plt.figure(figsize=(3, 5))
plt.bar(['Severity 0','Severity 1'],[rows_mv1_sv0+rows_mv2_sv0,rows_mv1_sv1+rows_mv2_sv1])

** The missing data seems randomly distributed, so we decided to go with the following strategy: **

* Drop rows with 2 NaN values

* Replace the NaN values from rows with 1 missing value

*First we get the mode of every feature for each class*

In [ ]:
mode_sv0 = data[data['Severity'] == 0].mode()
mode_sv1 = data[data['Severity'] == 1].mode()
mode_sv0 = mode_sv0.drop([1])
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(mode_sv0)
    print(mode_sv1)

*After we create conditions to replace the NaN values on rows with 1 missing value.*

*For that we need the index of the rows which have 1 missing value.*

In [ ]:
rows_1nan = missing_values_data_rows.index[missing_values_data_rows['data_missing'] == 1].tolist()
mask_sv0 = (data['Severity'] == 0) & (data.index.isin(rows_1nan))
mask_sv1 = (data['Severity'] == 1) & (data.index.isin(rows_1nan))

*We can now proceed and replace the missing values for their class mode* 

In [ ]:
data.loc[mask_sv0, 'Shape'] = data.loc[mask_sv0, 'Shape'].fillna(mode_sv0.loc[0,'Shape'])
data.loc[mask_sv0, 'Margin'] = data.loc[mask_sv0, 'Margin'].fillna(mode_sv0.loc[0,'Margin'])
data.loc[mask_sv0, 'Density'] = data.loc[mask_sv0, 'Density'].fillna(mode_sv0.loc[0,'Density'])
data.loc[mask_sv1, 'Age'] = data.loc[mask_sv1, 'Age'].fillna(mode_sv1.loc[0,'Age'])
data.loc[mask_sv1, 'Shape'] = data.loc[mask_sv1, 'Shape'].fillna(mode_sv1.loc[0,'Shape'])
data.loc[mask_sv1, 'Margin'] = data.loc[mask_sv1, 'Margin'].fillna(mode_sv1.loc[0,'Margin'])
data.loc[mask_sv1, 'Density'] = data.loc[mask_sv1, 'Density'].fillna(mode_sv1.loc[0,'Density'])
data

*Finally, we can drop rows with NaN values because the only ones that are left are the ones with 2 NaN*

In [ ]:
data = data.dropna()
data.index = np.arange(1, len(data) + 1)
data

In [ ]:
data.describe()

## Exploratory Data Analysis

https://towardsdatascience.com/a-starter-pack-to-exploratory-data-analysis-with-python-pandas-seaborn-and-scikit-learn-a77889485baf#249d

https://towardsdatascience.com/comprehensive-guide-to-exploratory-data-analysis-of-habermans-survival-data-set-b33f0373c83a

** Auxiliar functions & General definitions **

In [ ]:
c_palette = ['tab:green','tab:red']

def categorical_summarized(dataframe, x=None, y=None, hue=None, palette='Set1', verbose=True):
    if x == None:
        column_interested = y
    else:
        column_interested = x
    series = dataframe[column_interested]
    print(series.describe())
    print('mode: ', series.mode())
    if verbose:
        print('='*80)
        print(series.value_counts())

    sns.countplot(x=x, y=y, hue=hue, data=dataframe, palette=palette)
    plt.show()

def quantitative_summarized(dataframe, x=None, y=None, hue=None, palette='Set1', ax=None, verbose=True, swarm=False):
    series = dataframe[y]
    print(series.describe())
    print('mode: ', series.mode())
    if verbose:
        print('='*80)
        print(series.value_counts())

    sns.boxplot(x=x, y=y, hue=hue, data=dataframe, palette=palette, ax=ax)

    if swarm:
        sns.swarmplot(x=x, y=y, hue=hue, data=dataframe,
                      palette=palette, ax=ax)

    plt.show()

** Countplot of the Severity (Benign 0 vs Malignant 1) **

In [ ]:
sns.set_style('whitegrid')
ax = sns.countplot(x='Severity',data=data,palette=c_palette)


total = len(data['Severity'])

for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:.1f}%'.format(100 * height/total),
            ha="center") 

** Severity on Age **

In [ ]:
quantitative_summarized(dataframe= data, y = 'Age', x = 'Severity', palette=c_palette, verbose=False, swarm=True)

In [ ]:
sns.set_style('whitegrid')
sns.FacetGrid(data, hue='Severity', size=8, palette=c_palette) \
    .map(sns.distplot, 'Age', bins=10) \
    .add_legend()
plt.xticks([0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100])
plt.show()

In [ ]:
sns.set_style('darkgrid')
g = sns.FacetGrid(data,hue='Severity',palette=c_palette,size=6,aspect=2)
g = g.map(plt.hist,'Age',bins=20,alpha=0.7).add_legend()
plt.xticks([15,18,22,25,28,32,35,38,42,45,48,52,55,59,62,65,69,72,75,79,82,86,89,93,96,100])
plt.show()

In [ ]:
sns.jointplot(x='Age', y='Severity', data=data, kind='kde')
plt.show()

** Severity on Shape (mass shape: round=1 oval=2 lobular=3 irregular=4) **

In [ ]:
categorical_summarized(data, y = 'Shape', hue='Severity', palette=c_palette)

In [ ]:
sns.violinplot(data = data, x='Severity', y='Shape', palette=c_palette)
sns.swarmplot(data = data, x='Severity', y='Shape', color = 'k', alpha = 0.6, palette=c_palette)

** Severity on Margin (mass shape: round=1 oval=2 lobular=3 irregular=4) **

In [ ]:
categorical_summarized(data, y = 'Margin', hue='Severity', palette=c_palette)

In [ ]:
sns.violinplot(data = data, x='Severity', y='Margin', palette=c_palette)
sns.swarmplot(data = data, x='Severity', y='Margin', color = 'k', alpha = 0.6, palette=c_palette)

** Severity on Density (mass density high=1 iso=2 low=3 fat-containing=4) **

In [ ]:
categorical_summarized(data, y = 'Density', hue='Severity', palette=c_palette)

In [ ]:
sns.violinplot(data = data, x='Severity', y='Density', palette=c_palette)
sns.swarmplot(data = data, x='Severity', y='Density', color = 'k', alpha = 0.6, palette=c_palette)

##### Detect outliers: https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba

### Detect Outliers using Box plot (Uni-variate outlier)

In [ ]:
sns.boxplot(x=data['Age'])

In [ ]:
sns.boxplot(x=data['Shape'])

In [ ]:
sns.boxplot(x=data['Margin'])

In [ ]:
sns.boxplot(x=data['Density'])

### Detect Outliers using Scatter plot (Multi-variate outlier)

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))
ax.scatter(data['Age'], data['Shape'])
ax.set_xlabel('Age')
ax.set_ylabel('Shape')
#ax.set_ylabel('Margin')
#ax.set_ylabel('Density')
plt.show()

### Detect outliers using mathematical function Z-Score

In [ ]:
z = np.abs(stats.zscore(data))
threshold = 3
print(np.where(z > threshold))
# The first array contains the list of row numbers and second array respective column numbers

Column 3 (density) has all outliers

### Detect outliers using IQR Score
Similar to Z-Score

In [ ]:
Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)
iqr = Q3 - Q1
print(iqr)

In [ ]:
# Não curti ...
#print(data < (Q1 - 1.5 * iqr)) |(data > (Q3 + 1.5 * iqr))

## Data Preparation

### Remove Outliers using Z-Score

##### + explanations: https://stackoverflow.com/questions/23199796/detect-and-exclude-outliers-in-pandas-data-frame

In [ ]:
# Só fazer 1 vez
data = data[(np.abs(stats.zscore(data)) < 3).all(axis=1)]
data.index = np.arange(1, len(data) + 1)
data

### Converting pandas dataframes to numpy arrays

In [ ]:
X = data.drop('Severity',axis=1).to_numpy()
y = data['Severity'].to_numpy()
X

### Normalizing the attribute data using StandardScaler

** Fit scaler to the features **

In [ ]:
# Só fazer 1 vez
scaler = StandardScaler()
scaler.fit(X)
scaler.mean_

** Transform the features to a scaled version **

In [ ]:
X = scaler.transform(X)
X

## Train Test Split

** Create training and testing sets of the data **

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train

## Neural Networks & Genetic Algorithms

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# Função que contrói o modelo ANN
def buildModel(hidden_layers, nodes_per_layer, activation_fn, optimizer, loss_fn, metrics, inputs=4,):
    model = Sequential()
    #add input layer
    model.add(Dense(inputs, activation=activation_fn, input_shape=(inputs,)))

    #add hidden layers    
    for i in range(hidden_layers):
        model.add(Dense(nodes_per_layer, activation=activation_fn))

    #add output layer
    model.add(Dense(1,activation=activation_fn))

    #compile model
    model.compile(optimizer=optimizer, loss=loss_fn, metrics=metrics)

    return model

#Função que cria a população inicial
#parameters=[hidden_layers,nodes_per_layer,activation_fn]
def create_new_population():
    
    population=[]
    
    for i in range(10):
        cromo=[]
        cromo.append(np.random.randint(low=1, high=20))
        cromo.append(np.random.choice([1, 2, 4, 8, 16, 32, 64, 128, 256]))
        cromo.append(np.random.randint(low=0, high=4))
        population.append(cromo)
        
    return np.array(population)

#Função genérica que atualiza os argumentos do classifier
#parameters=[hidden_layers,nodes_per_layer,activation_fn]
def update_model_parameters(parameters):
    
    if((parameters[2]) == 0): a_f = 'relu'
    if((parameters[2]) == 1): a_f = 'softmax'
    if((parameters[2]) == 2): a_f = 'sigmoid'
    if((parameters[2]) == 3): a_f = 'tanh'
        
    model = buildModel(parameters[0],parameters[1],a_f,optimizer='rmsprop', loss_fn='binary_crossentropy', metrics=['accuracy'])

    print(type(a_f))
    return model

def select_mating_pool(pop, fitness, parents_fitness, num_parents):
    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
    parents = np.empty((num_parents, pop.shape[1]))
    #parents_fitness=[] - strangely not working good
    for parent_num in range(num_parents):
        #save fitness values of best parents
        parents_fitness.append(np.max(fitness))
        #save best parents
        max_fitness_idx = np.where(fitness == np.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num, :] = pop[max_fitness_idx, :]
        fitness[max_fitness_idx] = -99999999999
        
    return parents

def crossover(parents, offspring_size):
    offspring = np.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually it is at the center.
    crossover_point = np.uint8(offspring_size[1]/2)

    for k in range(offspring_size[0]):
        # Index of the first parent to mate.
        parent1_idx = k%parents.shape[0]
        # Index of the second parent to mate.
        parent2_idx = (k+1)%parents.shape[0]
        # The new offspring will have its first half of its genes taken from the first parent.
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        # The new offspring will have its second half of its genes taken from the second parent.
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring

def mutation(offspring_crossover):
    # Mutation changes a single gene in each offspring randomly.
    for idx in range(offspring_crossover.shape[0]):
        
        # Select which gene to mutate
        select_gene = np.random.randint(low=0, high=3)
 
        if(select_gene == 0):
            #num_hidden_layers mutation
            random_value = np.random.randint(low=1, high=20)
            offspring_crossover[idx,0] = random_value
        if(select_gene == 1):
            #num_nodes_per_layer mutation
            random_value = np.random.choice([1, 2, 4, 8, 16, 32, 64, 128, 256])
            offspring_crossover[idx,1] = random_value
        if(select_gene == 2):
            #activation function mutation
            random_value = np.random.randint(low=0, high=4)
            offspring_crossover[idx,2] = random_value
            
    return offspring_crossover

** K-Fold Cross Validation **

In [ ]:
from sklearn.model_selection import KFold

num_folds = 10
#acc_per_fold = []
#loss_per_fold = []

# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

** Keras ANN built from specific parameters (one specific chromosome) **

In [ ]:
def classify_create_folds(chromosome):
    scores=[]
    
    model = update_model_parameters(chromosome)
    
    fold_no=1
    
    for train, test in kfold.split(X, y):

        history = model.fit(X[train], y[train],
              epochs=20,
              batch_size=128,
              verbose=0)

        score = model.evaluate(X[test], y[test], batch_size=128, verbose=0)

        #acc_per_fold.append(score[1] * 100)
        #loss_per_fold.append(score[0])
        
        #Adding all accuracy values to an array
        scores.append(score[1])
    
        print("Fold %d: loss = %.2f || accuracy=  %.2f%%" % (fold_no, score[0], score[1]*100))

        fold_no+=1
        
    return scores

    #print('Average scores for all folds:')
    #print("> Accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(acc_per_fold),np.std(acc_per_fold)))
    #print("> Loss: %.2f " % (np.mean(loss_per_fold)))

** Hyperparameter optimization using Genetic Algorithms **

In [ ]:
population = create_new_population()
print(population)

num_parents_mating = 5
num_generations = 20
# number of genes for each chromosome
num_genes = 3
# number of chromosomes for each population
num_chromosomes = 10 

pop_size=(num_chromosomes,num_genes)

# fitness values for each chromosome for the current generation
fitness_values = []
# fitness vaalues for each chromosome of the last generation
last_fitness_values = []

gen = 0
cromo = 0

parents=[]
# Parents fitness so we do not repeat calculations on parents
parents_fitness = []

performances=[]
hiperparameters=[]


for generation in range(num_generations):
    gen+=1
    cromo = 0
    best_perf_per_gen = -1
    
    for chromosome in population:
        cromo+=1
        score=-1
        parentNumber=0
        
        # If it's a known chromosome we dont need to train the ANN again
        # Skips the first generation because we didnt select the parents yet
        for savedCromo in parents:
            parentNumber+=1
            if (np.array_equal(chromosome,savedCromo)):
                score = parents_fitness[parentNumber-1]
                print(savedCromo, "Known Chromosome")
                print(score, "Score Chromosome")

        
        # If it's a new chromosome we need to train the ANN in order to get the accuracy
        if (score < 0):
            scores = classify_create_folds(chromosome)
            score = sum(scores)/len(scores)
            
        print("Generation-{}".format(gen),"Chromosome-{}".format(cromo),"scored",score)
        # Keep the scores in fitness_values
        fitness_values.append(score)
        
        # Getting the best hyperparameters per generation to check the evolution at the end
        if(best_perf_per_gen < score):
            best_perf_per_gen = score
            best_cromo_per_gen = chromosome
           
        print(chromosome)
        
    performances.append(best_perf_per_gen)
    hiperparameters.append(best_cromo_per_gen)
   
    
    print(performances,"Best accuracies of each generation")
    print(hiperparameters,"Best of each generation")
    
    # We store last generation in other array because fitness_values is changed by the selec_mating_pool
    if(gen == num_generations):
        for i in fitness_values:
            last_fitness_values.append(i)
            
    print(last_fitness_values,"Last Fitness Values")
    
    parents_fitness = []
    parents = select_mating_pool(population,fitness_values,parents_fitness,num_parents_mating)

    # Generating next generation using crossover.
    offspring_crossover = crossover(parents,
                                        offspring_size=(pop_size[0]-parents.shape[0], num_genes))


    # Adding some variations to the offspring using mutation.
    offspring_mutation = mutation(offspring_crossover)

    # Creating the new population based on the parents and offspring.
    population[0:parents.shape[0], :] = parents
    population[parents.shape[0]:, :] = offspring_mutation
    
    # Reset fitness_values
    fitness_values=[]

# Getting the best solution
print(population)
best_solution = population[last_fitness_values.index(np.max(last_fitness_values))]
print("The best hyperparameters obtained are",best_solution,"with an accuracy of",np.max(last_fitness_values))

In [ ]:
'''
testModel = buildModel(hidden_layers=3, nodes_per_layer=32, activation_fn='relu',  optimizer='rmsprop', loss_fn='binary_crossentropy', metrics=['accuracy'])

history = testModel.fit(X_train, y_train,
          #epochs=20,
          #batch_size=128, validation_split=0.2)

score = testModel.evaluate(X_train, y_train, batch_size=128, verbose=1)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

# Visualize history
# Plot history: Loss
plt.plot(history.history['loss'])
plt.title('Validation loss history')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.show()

# Plot history: Accuracy
plt.plot(history.history['accuracy'])
plt.title('Validation accuracy history')
plt.ylabel('Accuracy value (%)')
plt.xlabel('No. epoch')
plt.show()
'''

In [ ]:
'''
from sklearn.model_selection import KFold
import numpy as np

num_folds = 10
acc_per_fold = []
loss_per_fold = []

# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

fold_no=1
for train, test in kfold.split(X, y):
    testModel = buildModel(hidden_layers=3, nodes_per_layer=32, activation_fn='relu',  optimizer='rmsprop', loss_fn='binary_crossentropy', metrics=['accuracy'])

    history = testModel.fit(X[train], y[train],
          epochs=20,
          batch_size=128,
          verbose=0)

    score = testModel.evaluate(X[test], y[test], batch_size=128, verbose=0)

    
    acc_per_fold.append(score[1] * 100)
    loss_per_fold.append(score[0])
    
    print("Fold %d: loss = %.2f || accuracy=  %.2f%%" % (fold_no, score[0], score[1]*100))

    fold_no+=1

print('Average scores for all folds:')
print("> Accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(acc_per_fold),np.std(acc_per_fold)))
print("> Loss: %.2f " % (np.mean(loss_per_fold)))
'''